In [1]:
import sympy as sp
import numpy as np
from IPython.display import display, Math, Latex
import codecs
import sage.all as sg

# Introduction

In this notebook a step-by-step symbolic solution to obtain the kernel $K(x|u)$ for the problem

$f(x)(1+W(x) + W(x+1)) - W(x)f(x-1) - W(x+1)f(x+1) = g(x)$

when  the weights are given by a barrier at $x= 0$ with 

$W(x) = \left\{\begin{matrix}
\lambda\,, & x\neq 0\\
\mu, & x=0\\
\end{matrix}\right.$

The kernel is shown to be $K(x|u) = a(x)a(u)e^{-|t(x)-t(u)|}$, where $t(x)$ is given implicitly by its derivative $\Delta t(x) = t(x)-t(x-1)$. We compute explicit solutions for $a(x)$ and $\Delta t(x)$.

# Homogeneous solution

The solution to the homogeneous problem with weights all equal to $\lambda$ is given by $\Delta t(x) = \log(r(\lambda))$. Where

$r(\lambda) =(1 + 2\lambda + \sqrt{4 \lambda +1})/(2\lambda)$

This means we can write $\lambda$ in terms of $r(\lambda)$ as

$\lambda = \frac{r(\lambda)}{(r(\lambda)-1)^2}$

This substitution will be useful later

In [2]:
x= sp.symbols('x')
l_inv = x/((x-1)**2)
r_fun = (1 + 2*x + sp.sqrt(4*x+1))/(2*x)

# General solution for the Barrier at $x_0 = 0$

We begin by defining the weight function W(x)

In [3]:
l= sp.symbols('l', positive = True)
mu= sp.symbols('mu', positive = True)
n = sp.symbols('n')

rmu = sp.symbols('rmu', positive = True)
rl= sp.symbols('rl', positive = True)

W = sp.Piecewise(
    (l_inv.subs(x,rl), x < 0),
    (l_inv.subs(x,rmu), sp.Eq(x,0)),
    (l_inv.subs(x,rl), x > 0),
)

display(Latex(r"$W(x) = $ " + sp.latex(W, mode='inline')))

<IPython.core.display.Latex object>

# Computing B(x)

$B(x-k)$ for $k>=0$ can be written in terms of $B(x) = \frac{p}{q}$ with

$  \frac{\text{numerator}(B(x-k))}{\text{denominator}(B(x-k))}  =   \begin{bmatrix}
1+2\lambda  & -\lambda^2\\ 
1 & 0
\end{bmatrix}^k
\begin{bmatrix}
p\\ 
r
\end{bmatrix} $

In [4]:
Bstep_mat = sp.Matrix([[1+2*l_inv.subs(x,rl), -l_inv.subs(x,rl)**2], [1, 0]])
k = sp.symbols('k')
bstart = sp.symbols('bs')
P, D = Bstep_mat.diagonalize()
Pinv = P.inv()

Bsteps_mat = (P * (D**k) * Pinv)*sp.Matrix([[bstart], [1]])
Bsteps_mat[0] = Bsteps_mat[0].simplify()
Bsteps_mat[1] = Bsteps_mat[1].simplify()

Bsteps = (Bsteps_mat[0]/Bsteps_mat[1]).simplify()
Bsteps

(rl**2 - 1)*(bs*rl**2 - 2*bs*rl - bs*rl**(2*k + 2) + 2*bs*rl**(2*k + 3) - bs*rl**(2*k + 4) + bs - rl**2 + rl**(2*k + 2))/((rl**4 - 2*rl**3 + 2*rl - 1)*(bs*rl**2 - 2*bs*rl - bs*rl**(2*k) + 2*bs*rl**(2*k + 1) - bs*rl**(2*k + 2) + bs - rl**2 + rl**(2*k)))

## B as a pure function

In [5]:
Bx1 = (l_inv.subs(x,rl)*rl).simplify()
Bx0 = ((1 + W.subs(x,0)+ W.subs(x,1) - W.subs(x,1)**2/Bx1)).simplify()
Bxm1 =  ((1 + W.subs(x,-1)+ W.subs(x,0) - W.subs(x,0)**2/Bx0)).simplify()
B_bwd_steps = Bsteps.replace(k,-x-1).replace(bstart, Bxm1).simplify()

B = sp.Piecewise(
    (Bx1, x > 0),
    (Bx0 , sp.Eq(x,0)), #at x==0
    (B_bwd_steps, x<0)
)

# Computing B*

B* can be computed by symmetry, only using a different b0

In [6]:
B_star_xm2 = (l_inv.subs(x,rl)*rl).simplify()
B_star_xm1 =  ((1 + W.subs(x,-1)+ W.subs(x,0) - W.subs(x,-1)**2/B_star_xm2)).simplify()
B_star_x0 =( (1 + W.subs(x,0)+ W.subs(x,1) - W.subs(x,0)**2/B_star_xm1)).simplify()
B_star_fwd_steps = Bsteps.replace(k,x).replace(bstart, B_star_x0).simplify()

B_star = sp.Piecewise(
    (B_star_xm2, x < -1),
    (B_star_xm1 , sp.Eq(x,-1)), #at x==-1
    (B_star_fwd_steps, x>-1)
)

# Computing a(x) and dt(x)

In [34]:
asqr = 1/(B- W*W/(B_star.replace(x,x-1)))
dt = sp.log((2*sp.sqrt(asqr.replace(x,x-1)*asqr)*W)/(-1 + sp.sqrt(4*asqr.replace(x,x-1)*asqr*W*W+1)))

We use the Sage library to perform simplifications on the square-root terms

In [36]:
dt_simplified = dt.subs(x,0)._sage_().simplify_full().canonicalize_radical().simplify_log()._sympy_() #use SAGE to simplify expressions
dt_simplified = sp.exp(dt_simplified.subs({'rl':rl,'rmu':rmu})) #replace some variables in simpy
dt_simplified = dt_simplified.subs(rmu, r_fun.subs(x,mu))._sage_().canonicalize_radical().simplify()._sympy_() #use SAGE to simplify expressions
dt_simplified = (dt_simplified-1).simplify()+1
dt_simplified = sp.log(dt_simplified).subs('rl',rl)
#finally, using that rl = l*(rl-1)^2
dt_simplified = dt_simplified.subs(rl/(rl-1),l*(rl-1))
dt_simplified

log(l*(rl - 1)/mu + 1)

# Asymptotics

In [293]:
y= sp.var('y',positive = True)
dt_asympt = dt.subs(x,y+1).subs(y,x-1)#simplifying piecewise function to x>1
dt_asympt = sp.exp( dt_asympt - sp.log(rl)) #compute difference with limit log(rl)
dt_asympt = dt_asympt._sage_().canonicalize_radical().simplify_full()._sympy_().subs('rl',rl) #use SAGE to simplify square root terms
dt_asympt = (dt_asympt) ** 2#remove square root (will require division 1/2 outside log-term later to compensate)


#simplify numerator and denominator for asymptotic behavior
numer = sp.numer(dt_asympt).expand().subs({'rl':rl, 'x':x})
numer = sum([term for term in numer.as_ordered_terms() if term.has(x)]) #collect only non constant terms
numer = (numer/rl**(2*x)).simplify() #divide numerator and denominator

denom = sp.denom(dt_asympt).expand().subs({'rl':rl, 'x':x})
denom = (denom/rl**(2*x)).expand()

denom_x = sum([term for term in denom.as_ordered_terms() if term.has(x)]) #collect only non constant terms
denom_ct = sum([term for term in denom.as_ordered_terms() if not term.has(x)]) #collect only non constant terms
denom = denom_x.factor(rl^x) + 1*(denom_ct)
                             
dt_asympt = sp.log(numer/denom.expand())/2 #has to divide outside log to compensate the square-root removal

dt_asympt

log(rl*(rl*rmu**2 + rl - 2*rmu)/(-rl**4*rmu/rl**(2*x) + rl**3*rmu**2/rl**(2*x) + rl**3/rl**(2*x) + rl**2*rmu**2 + rl**2 - rl**2*rmu/rl**(2*x) - 2*rl*rmu))/2

Then dt_asympt indicates the behavior 

$ \underset{x\rightarrow-\infty}{\lim}|\Delta t(x)-\log(r(\lambda))| \sim \frac{1}{2}\log\left( 1 + c\,r(\lambda)^{-2|x-x_0|} \right)\,,$